# Importing required libraries

In [1]:
import requests
from datetime import datetime
import pandas as pd
import numpy as np
import io

from scipy import stats

from IPython.display import display
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


from sklearn import linear_model
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

#https://github.com/dianalam/yelp-classifier/blob/master/yelp-classification.ipynb
from sklearn.pipeline import Pipeline, make_pipeline
from collections import defaultdict

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize, scale, StandardScaler, Normalizer
from sklearn.grid_search import GridSearchCV


import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, export_graphviz

# Connect to the OEEU's API and download data

In [2]:
with open("tokenOEEU.txt") as file: 
    api_token = file.read()

In [3]:
getDataDegree=requests.get('https://datos.oeeu.org/api/grado/estudiantes/',headers={'Authorization':api_token}).content
dataDegree=pd.read_csv(io.StringIO(getDataDegree.decode('utf-8')), low_memory=False, sep=";")

In [4]:
display(dataDegree.head())

,estudiante_id,CCAASeneca,UC3M_CumplirseExpectativasProf,UC3M_ProcesosSeleccionAfines,UC3M_UtilidadBolsaEmpleo,UC3M_ValoracionProgramasPracticas,UC3M_ValoracionSalidasProf,UEMC_Emprender_id,UEMC_NumeroPracticasEmpresas,UEMC_SalidaProfesional_id,UGR_ComoConociste_id,UGR_ConocesServiciosUniversitarios,UGR_PasadoServiciosOrientacionEmpleo,UGR_QueServiciosVisitaste_id,UGR_SatisfaccionServicios,UJA_rasgosEmpresaDescripcion,UJA_rasgosEmpresa_id,ULPGC_DondePracticas,ULPGC_MesesDuracionPracticas,ULPGC_ParticipacionPracticas,ULPGC_SatisfaccionPracticas,ULPGC_TrabajoDespuesPracticasEmpresa,UMA_InvolucradoActualmente,UMA_InvolucradoFase1,UMA_InvolucradoFase2,UMA_RolLiderazgoFase1,UMA_RolLiderazoFase2,UMH_VidaAspectos,UMH_VidaCircunstancias,UMH_VidaConseguirCosas,UMH_VidaRepetir,UMH_VidaSatisfaccion,UNED_BuenaBase_AprenderTrabajo,UNED_BuenaBase_DesarrolloPersonal,UNED_BuenaBase_EmpezarTrabajar,UNED_BuenaBase_Emprender,UNED_BuenaBase_PerspectivasProfesionales,UNED_BuenaBase_TareasTrabajo,UNED_DesarrolloProfesional_Casualidad,UNED_DesarrolloProfesional_Esfuerzo,UNED_DesarrolloProfesional_Objetivos,UNED_DesarrolloProfesional_Otros,UNED_DesarrolloProfesional_OtrosDescripcion,UNED_DesarrolloProfesional_Titulacion,UNED_DesarrolloProfesional_Valores,UNED_DesarrolloProfesional_Vocacion,UNED_DesarrolloTrayectoriaProfesional_id,UNED_EtapaDesarrolloProfesional_id,UNED_ValoracionTrayectoriaProfesional_id,UR_CarenciasFormacionU,UR_PersonalImpideTrabajar,UR_satisfaccionContenidoTrabajo,UR_satisfaccionMejoraPromocionTrabajo,UR_satisfaccionRetribucionTrabajo,accesoReferencias_nivel,accesoReferencias_obtenidoUniversidad,adaptabilidadNuevasSituaciones_nivel,adaptabilidadNuevasSituaciones_nivelExigidoTrabajo,adaptabilidadNuevasSituaciones_obtenidoUniversidad,ambitoActuacionEmpresa_id,annoFinalizacionEstudios,annoNacimiento,aprovechamientoRRSSEncontrarTrabajo_nivel,aprovechamientoRRSSEncontrarTrabajo_obtenidoUniversidad,autoconocimiento_nivel,autoconocimiento_obtenidoUniversidad,capacidadAnalisis_nivel,capacidadAnalisis_nivelExigidoTrabajo,capacidadAnalisis_obtenidoUniversidad,capacidadAprendizajeAutonomo_nivel,capacidadAprendizajeAutonomo_nivelExigidoTrabajo,capacidadAprendizajeAutonomo_obtenidoUniversidad,capacidadAsumirResponsabilidades_nivel,capacidadAsumirResponsabilidades_nivelExigidoTrabajo,capacidadAsumirResponsabilidades_obtenidoUniversidad,capacidadCompletarCV_nivel,capacidadCompletarCV_obtenidoUniversidad,capacidadCritica_nivel,capacidadCritica_nivelExigidoTrabajo,capacidadCritica_obtenidoUniversidad,capacidadEncontrarOfertasLaborales_nivel,capacidadEncontrarOfertasLaborales_obtenidoUniversidad,capacidadGestionarPresion_nivel,capacidadGestionarPresion_nivelExigidoTrabajo,capacidadGestionarPresion_obtenidoUniversidad,capacidadOrganizacionPlanificacion_nivel,capacidadOrganizacionPlanificacion_nivelExigidoTrabajo,capacidadOrganizacionPlanificacion_obtenidoUniversidad,capacidadOrganizarPlanBusquedaEmpleo_nivel,capacidadOrganizarPlanBusquedaEmpleo_obtenidoUniversidad,capacidadResolucionProblemas_nivel,capacidadResolucionProblemas_nivelExigidoTrabajo,capacidadResolucionProblemas_obtenidoUniversidad,capacidadTomaDecisiones_nivel,capacidadTomaDecisiones_nivelExigidoTrabajo,capacidadTomaDecisiones_obtenidoUniversidad,capacidadTrabajarContextoInternacional_nivel,capacidadTrabajarContextoInternacional_nivelExigidoTrabajo,capacidadTrabajarContextoInternacional_obtenidoUniversidad,capacidadTrabajarContextoMulticultural_nivel,capacidadTrabajarContextoMulticultural_nivelExigidoTrabajo,capacidadTrabajarContextoMulticultural_obtenidoUniversidad,capacidadTrabajarEquipoInterdisciplinar_nivel,capacidadTrabajarEquipoInterdisciplinar_nivelExigidoTrabajo,capacidadTrabajarEquipoInterdisciplinar_obtenidoUniversidad,capacidadTrabajarEquipo_nivel,capacidadTrabajarEquipo_nivelExigidoTrabajo,capacidadTrabajarEquipo_obtenidoUniversidad,caracterTitulacion_id,ccaaTrabajo_id,cicloTitulacion_id,comprensionLectoraLenguaExtranjera_nivel,comprensionLectoraLenguaExtranjera_nivelExigid

In [5]:
dataDegree_backup = dataDegree.copy()

In [6]:
dataDegree = dataDegree_backup.copy()

In [7]:
display(dataDegree.shape)

(134129, 493)

# Data wrangling and cleaning

In [8]:
dataDegree = dataDegree.filter(['estudiante_id','CCAASeneca','accesoReferencias_nivel','accesoReferencias_obtenidoUniversidad','adaptabilidadNuevasSituaciones_nivel','adaptabilidadNuevasSituaciones_nivelExigidoTrabajo','adaptabilidadNuevasSituaciones_obtenidoUniversidad','ambitoActuacionEmpresa_id','annoFinalizacionEstudios','annoNacimiento','aprovechamientoRRSSEncontrarTrabajo_nivel','aprovechamientoRRSSEncontrarTrabajo_obtenidoUniversidad','autoconocimiento_nivel','autoconocimiento_obtenidoUniversidad','capacidadAnalisis_nivel','capacidadAnalisis_nivelExigidoTrabajo','capacidadAnalisis_obtenidoUniversidad','capacidadAprendizajeAutonomo_nivel','capacidadAprendizajeAutonomo_nivelExigidoTrabajo','capacidadAprendizajeAutonomo_obtenidoUniversidad','capacidadAsumirResponsabilidades_nivel','capacidadAsumirResponsabilidades_nivelExigidoTrabajo','capacidadAsumirResponsabilidades_obtenidoUniversidad','capacidadCompletarCV_nivel','capacidadCompletarCV_obtenidoUniversidad','capacidadCritica_nivel','capacidadCritica_nivelExigidoTrabajo','capacidadCritica_obtenidoUniversidad','capacidadEncontrarOfertasLaborales_nivel','capacidadEncontrarOfertasLaborales_obtenidoUniversidad','capacidadGestionarPresion_nivel','capacidadGestionarPresion_nivelExigidoTrabajo','capacidadGestionarPresion_obtenidoUniversidad','capacidadOrganizacionPlanificacion_nivel','capacidadOrganizacionPlanificacion_nivelExigidoTrabajo','capacidadOrganizacionPlanificacion_obtenidoUniversidad','capacidadOrganizarPlanBusquedaEmpleo_nivel','capacidadOrganizarPlanBusquedaEmpleo_obtenidoUniversidad','capacidadResolucionProblemas_nivel','capacidadResolucionProblemas_nivelExigidoTrabajo','capacidadResolucionProblemas_obtenidoUniversidad','capacidadTomaDecisiones_nivel','capacidadTomaDecisiones_nivelExigidoTrabajo','capacidadTomaDecisiones_obtenidoUniversidad','capacidadTrabajarContextoInternacional_nivel','capacidadTrabajarContextoInternacional_nivelExigidoTrabajo','capacidadTrabajarContextoInternacional_obtenidoUniversidad','capacidadTrabajarContextoMulticultural_nivel','capacidadTrabajarContextoMulticultural_nivelExigidoTrabajo','capacidadTrabajarContextoMulticultural_obtenidoUniversidad','capacidadTrabajarEquipoInterdisciplinar_nivel','capacidadTrabajarEquipoInterdisciplinar_nivelExigidoTrabajo','capacidadTrabajarEquipoInterdisciplinar_obtenidoUniversidad','capacidadTrabajarEquipo_nivel','capacidadTrabajarEquipo_nivelExigidoTrabajo','capacidadTrabajarEquipo_obtenidoUniversidad','caracterTitulacion_id','ccaaTrabajo_id','cicloTitulacion_id','comprensionLectoraLenguaExtranjera_nivel','comprensionLectoraLenguaExtranjera_nivelExigidoTrabajo','comprensionLectoraLenguaExtranjera_obtenidoUniversidad','comprension_aleman','comprension_chinoMandarin','comprension_frances','comprension_ingles','comprension_italiano','comprension_portugues','compromisoEticoTrabajo_nivel','compromisoEticoTrabajo_nivelExigidoTrabajo','compromisoEticoTrabajo_obtenidoUniversidad','comunicacionEscritaLenguaExtranjera_nivel','comunicacionEscritaLenguaExtranjera_nivelExigidoTrabajo','comunicacionEscritaLenguaExtranjera_obtenidoUniversidad','comunicacionEscrita_nivel','comunicacionEscrita_nivelExigidoTrabajo','comunicacionEscrita_obtenidoUniversidad','comunicacionOralLenguaExtranjera_nivel','comunicacionOralLenguaExtranjera_nivelExigidoTrabajo','comunicacionOralLenguaExtranjera_obtenidoUniversidad','comunicacionOral_nivel','comunicacionOral_nivelExigidoTrabajo','comunicacionOral_obtenidoUniversidad','conocimientoOportunidadesMercadoTrabajo_nivel','conocimientoOportunidadesMercadoTrabajo_obtenidoUniversidad','conocimientosHabilidadesAparatosElectronicos_nivel','conocimientosHabilidadesAparatosElectronicos_nivelExigidoTrabajo','conocimientosHabilidadesAparatosElectronicos_obtenidoUniversidad','conocimientosHabilidadesEdicionMultimedia_nivel','conocimientosHabilidadesEdicionMultimedia_nivelExigidoTrabajo','conocimientosHabilidadesEdicionMultimedia_obtenidoUniversidad','conocimientosHabilidadesOfimatica_nivel','conocimientosHabilidadesOfimatica_nivelExigidoTrabajo','conocimientosHabilidadesOfimatica_obtenidoUniversidad','conocimientosRelativosRedesSocialesInternet_nivel','conocimientosRelativosRedesSocialesInternet_nivelExigidoTrabajo','conocimientosRelativosRedesSocialesInternet_obtenidoUniversidad','conseguirUltimoTrabajo_anno_id','conseguirUltimoTrabajo_anno_original','conseguirUltimoTrabajo_mes_id','conseguirUltimoTrabajo_mes_original','conversacion_aleman','conversacion_chinoMandarin','conversacion_frances','conversacion_ingles','conversacion_italiano','conversacion_portugues','creatividad_nivel','creatividad_nivelExigidoTrabajo','creatividad_obtenidoUniversidad','cuestionarioFinalizado','cuestionarioIniciado','cursoFinalizacionEstudios','cursoInicioEstudios','dominioCompetenciasTitulacion_nivel','dominioCompetenciasTitulacion_nivelExigidoTrabajo','dominioCompetenciasTitulacion_obtenidoUniversidad','edad','edicion','elementoClaveConseguirEmpleo_CompetenciasEspecificas','elementoClaveConseguirEmpleo_CompetenciasGenericas','elementoClaveConseguirEmpleo_ExpedienteAcademico','elementoClaveConseguirEmpleo_ExperienciaLaboral','elementoClaveConseguirEmpleo_Idiomas','elementoClaveConseguirEmpleo_Otros','elementoClaveConseguirEmpleo_OtrosDescripcion','elementoClaveConseguirEmpleo_TICs','elementoClaveConseguirEmpleo_TitulacionEspecifica','elementoClaveConseguirEmpleo_Titulo','elementoClaveConseguirPrimerEmpleo_CompetenciasEspecificas','elementoClaveConseguirPrimerEmpleo_CompetenciasGenericas','elementoClaveConseguirPrimerEmpleo_ExpedienteAcademico','elementoClaveConseguirPrimerEmpleo_ExperienciaLaboral','elementoClaveConseguirPrimerEmpleo_Idiomas','elementoClaveConseguirPrimerEmpleo_Otros','elementoClaveConseguirPrimerEmpleo_OtrosDescripcion','elementoClaveConseguirPrimerEmpleo_TICs','elementoClaveConseguirPrimerEmpleo_TitulacionEspecifica','elementoClaveConseguirPrimerEmpleo_Titulo','email','emailRequerido','empresaEsEspannola','esEspannol','escritura_aleman','escritura_chinoMandarin','escritura_frances','escritura_ingles','escritura_italiano','escritura_portugues','estudiosMadre_cod','estudiosMadre_id','estudiosPadre_cod','estudiosPadre_id','experienciaLaboralDuranteEstudios','haEstadoDesempleado','habilidadTrabajoIndependiente_nivel','habilidadTrabajoIndependiente_nivelExigidoTrabajo','habilidadTrabajoIndependiente_obtenidoUniversidad','habilidadesEntrevista_nivel','habilidadesEntrevista_obtenidoUniversidad','habilidadesGestionInformacion_nivel','habilidadesGestionInformacion_nivelExigidoTrabajo','habilidadesGestionInformacion_obtenidoUniversidad','habilidadesInterpersonales_nivel','habilidadesInterpersonales_nivelExigidoTrabajo','habilidadesInterpersonales_obtenidoUniversidad','habilidadesNavegacionInternet_nivel','habilidadesNavegacionInternet_nivelExigidoTrabajo','habilidadesNavegacionInternet_obtenidoUniversidad','habitantesDomicilio_id','hijosMenores_id','importanciaAprendizajeBasadoProblemas','importanciaAsistenciaClase','importanciaConocimientosPracticosMetodologicos','importanciaExposicionesOrales','importanciaParticipacionProyectosInvestigacion','importanciaPracticasEmpresasInstituciones','importanciaProfesorFuenteInformacion','importanciaTeoriasConceptosParadigmas','importanciaTrabajosEscritos','importanciaTrabajosGrupo','iniciativaPersonal_nivel','iniciativaPersonal_nivelExigidoTrabajo','iniciativaPersonal_obtenidoUniversidad','innovacion_nivel','innovacion_nivelExigidoTrabajo','innovacion_obtenidoUniversidad','liderazgo_nivel','liderazgo_nivelExigidoTrabajo','liderazgo_obtenidoUniversidad','mesFinalizacionEstudios_id','mesesResidenciaExtranjeroDespuesEstudios','mesesResidenciaExtranjeroDespuesEstudios_cod','mesesResidenciaExtranjeroDespuesEstudios_estudios','mesesResidenciaExtranjeroDespuesEstudios_estudiosCod','mesesResidenciaExtranjeroDespuesEstudios_trabajo','mesesResidenciaExtranjeroDespuesEstudios_trabajoCod','mesesResidenciaExtranjeroDuranteEstudios','mesesResidenciaExtranjeroDuranteEstudios_estudios','mesesResidenciaExtranjeroDuranteEstudios_estudiosCod','mesesResidenciaExtranjeroDuranteEstudios_trabajo','mesesResidenciaExtranjeroDuranteEstudios_trabajoCod','mesesTrabajoDuranteEstudios','metodoClaveEmpleo_AnuncioPrensa','metodoClaveEmpleo_AsociacionesExAlumnos','metodoClaveEmpleo_ColegioProfesional','metodoClaveEmpleo_ContactoDirecto','metodoClaveEmpleo_ContactoPersonal','metodoClaveEmpleo_ETTs','metodoClaveEmpleo_EmpresasSeleccionPersonal','metodoClaveEmpleo_Otros','metodoClaveEmpleo_PortalEmpleo','metodoClaveEmpleo_RRSS','metodoClaveEmpleo_ServiciosPublicosEmpleo','metodoClaveEmpleo_ServiciosUnivEmpleo','metodoClaveEmpleo_Sindicato','metodoClaveEmpleo_WebCorporativa','metodoClavePrimerEmpleo_AnuncioPrensa','metodoClavePrimerEmpleo_AsociacionesExAlumnos','metodoClavePrimerEmpleo_ColegioProfesional','metodoClavePrimerEmpleo_ContactoDirecto','metodoClavePrimerEmpleo_ContactoPersonal','metodoClavePrimerEmpleo_ETTs','metodoClavePrimerEmpleo_EmpresasSeleccionPersonal','metodoClavePrimerEmpleo_Otros','metodoClavePrimerEmpleo_PortalEmpleo','metodoClavePrimerEmpleo_RRSS','metodoClavePrimerEmpleo_ServiciosPublicosEmpleo','metodoClavePrimerEmpleo_ServiciosUnivEmpleo','metodoClavePrimerEmpleo_Sindicato','metodoClavePrimerEmpleo_WebCorporativa','metodoUsadoBusquedaEmpleo_AnuncioPrensa','metodoUsadoBusquedaEmpleo_AsociacionesExAlumnos','metodoUsadoBusquedaEmpleo_ColegioProfesional','metodoUsadoBusquedaEmpleo_ContactoDirecto','metodoUsadoBusquedaEmpleo_ContactoPersonal','metodoUsadoBusquedaEmpleo_ETTs','metodoUsadoBusquedaEmpleo_EmpresasSeleccionPersonal','metodoUsadoBusquedaEmpleo_Otros','metodoUsadoBusquedaEmpleo_PortalEmpleo','metodoUsadoBusquedaEmpleo_RRSS','metodoUsadoBusquedaEmpleo_ServiciosPublicosEmpleo','metodoUsadoBusquedaEmpleo_ServiciosUnivEmpleo','metodoUsadoBusquedaEmpleo_Sindicato','metodoUsadoBusquedaEmpleo_WebCorporativa','metodoUsadoBusquedaPrimerEmpleo_AnuncioPrensa','metodoUsadoBusquedaPrimerEmpleo_AsociacionesExAlumnos','metodoUsadoBusquedaPrimerEmpleo_ColegioProfesional','metodoUsadoBusquedaPrimerEmpleo_ContactoDirecto','metodoUsadoBusquedaPrimerEmpleo_ContactoPersonal','metodoUsadoBusquedaPrimerEmpleo_ETTs','metodoUsadoBusquedaPrimerEmpleo_EmpresasSeleccionPersonal','metodoUsadoBusquedaPrimerEmpleo_Otros','metodoUsadoBusquedaPrimerEmpleo_PortalEmpleo','metodoUsadoBusquedaPrimerEmpleo_RRSS','metodoUsadoBusquedaPrimerEmpleo_ServiciosPublicosEmpleo','metodoUsadoBusquedaPrimerEmpleo_ServiciosUnivEmpleo','metodoUsadoBusquedaPrimerEmpleo_Sindicato','metodoUsadoBusquedaPrimerEmpleo_WebCorporativa','motivacionCalidad_nivel','motivacionCalidad_nivelExigidoTrabajo','motivacionCalidad_obtenidoUniversidad','motivosJornadaLaboralTiempoParcialPrimerEmpleo_id','motivosJornadaLaboralTiempoParcial_id','nivelEstudiosAdecuadoPrimerTrabajo_id','nivelEstudiosAdecuadoUltimoTrabajo_id','notaMedia2','notaMedia_id','oficioProfesionMadre_cod','oficioProfesionMadre_cod2','oficioProfesionMadre_id','oficioProfesionPadre_cod','oficioProfesionPadre_cod2','oficioProfesionPadre_id','otroMetodoUsadoBusquedaEmpleo','otroMetodoUsadoBusquedaPrimerEmpleo','otroTituloActualidad','otroTituloActualidad_Cod','otroTituloActualidad_diplomatura','otroTituloActualidad_doctorado','otroTituloActualidad_fp','otroTituloActualidad_licenciatura','otroTituloActualidad_master','otroTituloUniversitario','otroTituloUniversitario_Cod','otroTituloUniversitario_diplomatura','otroTituloUniversitario_doctorado','otroTituloUniversitario_fp','otroTituloUniversitario_licenciatura','otroTituloUniversitario_master','otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal','otrosCriteriosSeleccionarPuestoTrabajo_geografica','otrosCriteriosSeleccionarPuestoTrabajo_otros','otrosCriteriosSeleccionarPuestoTrabajo_otrosDescripcion','otrosCriteriosSeleccionarPuestoTrabajo_prestigio','otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios','otrosCriteriosSeleccionarPuestoTrabajo_salario','otrosProgramasIntercambio','otrosRequisitosOptarPrimerTrabajo','otrosRequisitosOptarPuestoTrabajo','paisErasmus','paisResidenciaHabitual_id','paisTrabajo_id','participacionProgramasIntercambio','realizacionPracticasEstudios','realizacionPracticasExtracurriculares','realizacionPracticasObligatoriasCurriculares','realizacionPracticasOptativasCurriculares','relacionPrimerEmpleoEstudios','relacionTrabajoDuranteEstudiosTitulacion','requisitosOptarPrimerTrabajo_CompetenciasEspecificas','requisitosOptarPrimerTrabajo_CompetenciasGenericas','requisitosOptarPrimerTrabajo_ExpedienteAcademico','requisitosOptarPrimerTrabajo_ExperienciaLaboralPrevia','requisitosOptarPrimerTrabajo_Idiomas','requisitosOptarPrimerTrabajo_Otros','requisitosOptarPrimerTrabajo_TIC','requisitosOptarPrimerTrabajo_TitulacionEspecifica','requisitosOptarPrimerTrabajo_TituloUniversitario','requisitosOptarPuestoTrabajo_CompetenciasEspecificas','requisitosOptarPuestoTrabajo_CompetenciasGenericas','requisitosOptarPuestoTrabajo_ExpedienteAcademico','requisitosOptarPuestoTrabajo_ExperienciaLaboralPrevia','requisitosOptarPuestoTrabajo_Idiomas','requisitosOptarPuestoTrabajo_Otros','requisitosOptarPuestoTrabajo_TIC','requisitosOptarPuestoTrabajo_TitulacionEspecifica','requisitosOptarPuestoTrabajo_TituloUniversitario','residenciaEspanna','residenciaExtranjeroDespuesEstudios','residenciaExtranjeroDespuesEstudios_estudios','residenciaExtranjeroDespuesEstudios_trabajo','residenciaExtranjeroDuranteEstudios','residenciaExtranjeroDuranteEstudiosMotivos_cod','residenciaExtranjeroDuranteEstudios_estudios','residenciaExtranjeroDuranteEstudios_trabajo','residenciaExtranjeroMotivos_cod','residenciaFamiliar_id','residenciaHabitual_id','residencia_id','satisfaccionContenidosTitulacion','satisfaccionFormacionRecibidaGeneral','satisfaccionMetodologiasEducativas','satisfaccionProfesorado','satisfaccionServiciosApoyoEstudiante','sensibilidadTemasMedioambientalesSociales_nivel','sensibilidadTemasMedioambientalesSociales_nivelExigidoTrabajo','sensibilidadTemasMedioambientalesSociales_obtenidoUniversidad','sexo_id','situacionLaboralEmpleo_Otros','situacionLaboralEmpleo_id','situacionLaboralMadre_cod','situacionLaboralMadre_id','situacionLaboralPadre_cod','situacionLaboralPadre_id','situacionLaboralPrimerEmpleo_Otros','situacionLaboralRelacionEstudios_id','tiempoDuracionPracticasCurricularesObligatorias_Cod','tiempoDuracionPracticasCurricularesOptativas_Cod','tiempoDuracionPracticasExtracurriculares','tiempoDuracionPracticasExtracurriculares_Cod','tiempoDuracionPracticasObligatoriasCurriculares','tiempoDuracionPracticasOptativasCurriculares','tiempoDuracionPracticas_id','tiempoErasmus','tiempoExtraTitulacion','tiempoExtranjeroMotivoEstudios_Cod','tiempoExtranjeroMotivoTrabajo_Cod','tiempoExtranjero_Cod','tiempoOficialTitulacion_id','tiempoRealizacionPracticas','tiempoSeneca','tieneHijos','tipoProgramaIntercambio_Cod','tipoProgramaIntercambio_id','tipoTitulacion_id','tipologiaPracticas','titulacion_id','trabajoDuranteEstudios_Bool','trabajoDuranteEstudios_id','universidad_id','viaAccesoUniversidad_id','volverACursarEstudios_id'], axis=1)

In [9]:
display(dataDegree.shape)

(134129, 383)

In [10]:
dataDegree = dataDegree[pd.notnull(dataDegree['haEstadoDesempleado'])]

print("Shape of the dataframe with only users have their employment status (anytime) not undefined:"+str(dataDegree.shape))

Shape of the dataframe with only users have their employment status (anytime) not undefined:(9744, 383)


# Cleaning columns with too much NaN values

Data cleaning and wrangling

In [11]:
NaNValues = dataDegree.isnull().sum().tolist()
columns = dataDegree.columns.values.tolist()

print("Total rows: ", dataDegree['estudiante_id'].count())

print("\nTotal NaN values per column\n----------")
i=0
for c in columns:
    print(str(columns[i])+": "+str(NaNValues[i]))
    i+=1

Total rows:  9744

Total NaN values per column
----------
estudiante_id: 0
CCAASeneca: 9714
accesoReferencias_nivel: 382
accesoReferencias_obtenidoUniversidad: 412
adaptabilidadNuevasSituaciones_nivel: 217
adaptabilidadNuevasSituaciones_nivelExigidoTrabajo: 690
adaptabilidadNuevasSituaciones_obtenidoUniversidad: 222
ambitoActuacionEmpresa_id: 969
annoFinalizacionEstudios: 1366
annoNacimiento: 696
aprovechamientoRRSSEncontrarTrabajo_nivel: 318
aprovechamientoRRSSEncontrarTrabajo_obtenidoUniversidad: 361
autoconocimiento_nivel: 318
autoconocimiento_obtenidoUniversidad: 345
capacidadAnalisis_nivel: 313
capacidadAnalisis_nivelExigidoTrabajo: 815
capacidadAnalisis_obtenidoUniversidad: 320
capacidadAprendizajeAutonomo_nivel: 176
capacidadAprendizajeAutonomo_nivelExigidoTrabajo: 661
capacidadAprendizajeAutonomo_obtenidoUniversidad: 180
capacidadAsumirResponsabilidades_nivel: 242
capacidadAsumirResponsabilidades_nivelExigidoTrabajo: 718
capacidadAsumirResponsabilidades_obtenidoUniversidad: 258

In [12]:
totalRows = dataDegree['estudiante_id'].count()

# max percentage of NaN values allowed in a column
NaNThreshold = 10

NaNValues = dataDegree.isnull().sum().tolist()
columns = dataDegree.columns.values.tolist()

i=0
for c in columns:
    if dataDegree[columns[i]].isnull().sum() > NaNThreshold:
        print("Drop column: ",columns[i])
        dataDegree.drop([columns[i]], axis=1, inplace=True)
    i+=1

display(dataDegree.head())

Drop column:  CCAASeneca
Drop column:  accesoReferencias_nivel
Drop column:  accesoReferencias_obtenidoUniversidad
Drop column:  adaptabilidadNuevasSituaciones_nivel
Drop column:  adaptabilidadNuevasSituaciones_nivelExigidoTrabajo
Drop column:  adaptabilidadNuevasSituaciones_obtenidoUniversidad
Drop column:  ambitoActuacionEmpresa_id
Drop column:  annoFinalizacionEstudios
Drop column:  annoNacimiento
Drop column:  aprovechamientoRRSSEncontrarTrabajo_nivel
Drop column:  aprovechamientoRRSSEncontrarTrabajo_obtenidoUniversidad
Drop column:  autoconocimiento_nivel
Drop column:  autoconocimiento_obtenidoUniversidad
Drop column:  capacidadAnalisis_nivel
Drop column:  capacidadAnalisis_nivelExigidoTrabajo
Drop column:  capacidadAnalisis_obtenidoUniversidad
Drop column:  capacidadAprendizajeAutonomo_nivel
Drop column:  capacidadAprendizajeAutonomo_nivelExigidoTrabajo
Drop column:  capacidadAprendizajeAutonomo_obtenidoUniversidad
Drop column:  capacidadAsumirResponsabilidades_nivel
Drop column:

Drop column:  importanciaConocimientosPracticosMetodologicos
Drop column:  importanciaExposicionesOrales
Drop column:  importanciaParticipacionProyectosInvestigacion
Drop column:  importanciaPracticasEmpresasInstituciones
Drop column:  importanciaProfesorFuenteInformacion
Drop column:  importanciaTeoriasConceptosParadigmas
Drop column:  importanciaTrabajosEscritos
Drop column:  importanciaTrabajosGrupo
Drop column:  iniciativaPersonal_nivel
Drop column:  iniciativaPersonal_nivelExigidoTrabajo
Drop column:  iniciativaPersonal_obtenidoUniversidad
Drop column:  innovacion_nivel
Drop column:  innovacion_nivelExigidoTrabajo
Drop column:  innovacion_obtenidoUniversidad
Drop column:  liderazgo_nivel
Drop column:  liderazgo_nivelExigidoTrabajo
Drop column:  liderazgo_obtenidoUniversidad
Drop column:  mesFinalizacionEstudios_id
Drop column:  mesesResidenciaExtranjeroDespuesEstudios
Drop column:  mesesResidenciaExtranjeroDespuesEstudios_cod
Drop column:  mesesResidenciaExtranjeroDespuesEstudios_

Drop column:  viaAccesoUniversidad_id
Drop column:  volverACursarEstudios_id


,estudiante_id,cicloTitulacion_id,cuestionarioFinalizado,cuestionarioIniciado,cursoFinalizacionEstudios,edicion,emailRequerido,haEstadoDesempleado,otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal,otrosCriteriosSeleccionarPuestoTrabajo_geografica,otrosCriteriosSeleccionarPuestoTrabajo_otros,otrosCriteriosSeleccionarPuestoTrabajo_prestigio,otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios,otrosCriteriosSeleccionarPuestoTrabajo_salario,residenciaExtranjeroDespuesEstudios,residenciaExtranjeroDespuesEstudios_estudios,residenciaExtranjeroDespuesEstudios_trabajo,residenciaExtranjeroDuranteEstudios,residenciaExtranjeroDuranteEstudiosMotivos_cod,residenciaExtranjeroDuranteEstudios_estudios,residenciaExtranjeroDuranteEstudios_trabajo,residenciaExtranjeroMotivos_cod,sexo_id,tipologiaPracticas,titulacion_id,universidad_id
20,6409021,2,True,True,20092010,2015,True,True,False,True,False,False,True,True,False,False,False,False,5.0,False,False,5.0,2,4.0,5097000,64
21,6409022,2,True,True,20092010,2015,True,True,True,True,False,False,False,True,False,False,False,False,5.0,False,False,5.0,2,4.0,5117000,64
22,6409023,2,True,True,20092010,2015,True,True,False,True,False,False,False,True,True,False,True,True,3.0,True,False,2.0,2,4.0,4011000,64
38,6409039,3,True,True,20092010,2015,True,True,False,True,True,False,False,True,False,False,False,False,5.0,False,False,5.0,2,4.0,2500394,64
40,6409041,2,True,True,20092010,2015,True,True,False,False,True,False,True,False,False,False,False,False,5.0,False,False,5.0,1,4.0,5098000,64


In [13]:
display(dataDegree.shape)

(9744, 26)

In [14]:
display(dataDegree.head())

,estudiante_id,cicloTitulacion_id,cuestionarioFinalizado,cuestionarioIniciado,cursoFinalizacionEstudios,edicion,emailRequerido,haEstadoDesempleado,otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal,otrosCriteriosSeleccionarPuestoTrabajo_geografica,otrosCriteriosSeleccionarPuestoTrabajo_otros,otrosCriteriosSeleccionarPuestoTrabajo_prestigio,otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios,otrosCriteriosSeleccionarPuestoTrabajo_salario,residenciaExtranjeroDespuesEstudios,residenciaExtranjeroDespuesEstudios_estudios,residenciaExtranjeroDespuesEstudios_trabajo,residenciaExtranjeroDuranteEstudios,residenciaExtranjeroDuranteEstudiosMotivos_cod,residenciaExtranjeroDuranteEstudios_estudios,residenciaExtranjeroDuranteEstudios_trabajo,residenciaExtranjeroMotivos_cod,sexo_id,tipologiaPracticas,titulacion_id,universidad_id
20,6409021,2,True,True,20092010,2015,True,True,False,True,False,False,True,True,False,False,False,False,5.0,False,False,5.0,2,4.0,5097000,64
21,6409022,2,True,True,20092010,2015,True,True,True,True,False,False,False,True,False,False,False,False,5.0,False,False,5.0,2,4.0,5117000,64
22,6409023,2,True,True,20092010,2015,True,True,False,True,False,False,False,True,True,False,True,True,3.0,True,False,2.0,2,4.0,4011000,64
38,6409039,3,True,True,20092010,2015,True,True,False,True,True,False,False,True,False,False,False,False,5.0,False,False,5.0,2,4.0,2500394,64
40,6409041,2,True,True,20092010,2015,True,True,False,False,True,False,True,False,False,False,False,False,5.0,False,False,5.0,1,4.0,5098000,64


In [15]:
# Change numeric values per their categorical traduction in the proper cases (variables)

In [16]:
dataDegree = dataDegree.replace(False,0)
dataDegree = dataDegree.replace(True,1)

# Performing one-hot encoding for categorial values on columns

In [17]:
dataDegree_workDummies = pd.get_dummies(dataDegree)
display(dataDegree_workDummies.head())

,estudiante_id,cicloTitulacion_id,cuestionarioFinalizado,cuestionarioIniciado,cursoFinalizacionEstudios,edicion,emailRequerido,haEstadoDesempleado,otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal,otrosCriteriosSeleccionarPuestoTrabajo_geografica,otrosCriteriosSeleccionarPuestoTrabajo_otros,otrosCriteriosSeleccionarPuestoTrabajo_prestigio,otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios,otrosCriteriosSeleccionarPuestoTrabajo_salario,residenciaExtranjeroDespuesEstudios,residenciaExtranjeroDespuesEstudios_estudios,residenciaExtranjeroDespuesEstudios_trabajo,residenciaExtranjeroDuranteEstudios,residenciaExtranjeroDuranteEstudiosMotivos_cod,residenciaExtranjeroDuranteEstudios_estudios,residenciaExtranjeroDuranteEstudios_trabajo,residenciaExtranjeroMotivos_cod,sexo_id,tipologiaPracticas,titulacion_id,universidad_id
20,6409021,2,True,True,20092010,2015,True,1,0,1,0,0,1,1,0,0,0,0,5.0,0,0,5.0,2,4.0,5097000,64
21,6409022,2,True,True,20092010,2015,True,1,1,1,0,0,0,1,0,0,0,0,5.0,0,0,5.0,2,4.0,5117000,64
22,6409023,2,True,True,20092010,2015,True,1,0,1,0,0,0,1,1,0,1,1,3.0,1,0,2.0,2,4.0,4011000,64
38,6409039,3,True,True,20092010,2015,True,1,0,1,1,0,0,1,0,0,0,0,5.0,0,0,5.0,2,4.0,2500394,64
40,6409041,2,True,True,20092010,2015,True,1,0,0,1,0,1,0,0,0,0,0,5.0,0,0,5.0,1,4.0,5098000,64


In [18]:
dataDegree_workDummies = dataDegree_workDummies.dropna()

display(dataDegree_workDummies.head())

,estudiante_id,cicloTitulacion_id,cuestionarioFinalizado,cuestionarioIniciado,cursoFinalizacionEstudios,edicion,emailRequerido,haEstadoDesempleado,otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal,otrosCriteriosSeleccionarPuestoTrabajo_geografica,otrosCriteriosSeleccionarPuestoTrabajo_otros,otrosCriteriosSeleccionarPuestoTrabajo_prestigio,otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios,otrosCriteriosSeleccionarPuestoTrabajo_salario,residenciaExtranjeroDespuesEstudios,residenciaExtranjeroDespuesEstudios_estudios,residenciaExtranjeroDespuesEstudios_trabajo,residenciaExtranjeroDuranteEstudios,residenciaExtranjeroDuranteEstudiosMotivos_cod,residenciaExtranjeroDuranteEstudios_estudios,residenciaExtranjeroDuranteEstudios_trabajo,residenciaExtranjeroMotivos_cod,sexo_id,tipologiaPracticas,titulacion_id,universidad_id
20,6409021,2,True,True,20092010,2015,True,1,0,1,0,0,1,1,0,0,0,0,5.0,0,0,5.0,2,4.0,5097000,64
21,6409022,2,True,True,20092010,2015,True,1,1,1,0,0,0,1,0,0,0,0,5.0,0,0,5.0,2,4.0,5117000,64
22,6409023,2,True,True,20092010,2015,True,1,0,1,0,0,0,1,1,0,1,1,3.0,1,0,2.0,2,4.0,4011000,64
38,6409039,3,True,True,20092010,2015,True,1,0,1,1,0,0,1,0,0,0,0,5.0,0,0,5.0,2,4.0,2500394,64
40,6409041,2,True,True,20092010,2015,True,1,0,0,1,0,1,0,0,0,0,0,5.0,0,0,5.0,1,4.0,5098000,64


In [19]:
dataDegree_workDummies.drop(['estudiante_id'], axis=1, inplace=True)

# Random Forest Algorithm execution
## Preparing data: selecting data for training and testing from the dataset


In [20]:
dataDegree_workDummies['is_train'] = np.random.uniform(0,1,len(dataDegree_workDummies)) <= .66

In [21]:
train_gen, test_gen = dataDegree_workDummies[dataDegree_workDummies['is_train']==True], dataDegree_workDummies[dataDegree_workDummies['is_train']==False]

## Defining function to search the best setup for Random forest

In [22]:
def grid_search_scores_rf(Xtrain, ytrain):
    scores = ['accuracy', 'precision', 'roc_auc']
    output = defaultdict(list)
    pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())
    param = {
        'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
        'randomforestclassifier__min_samples_leaf': [1, 5, 10, 50, 100, 200, 500, 1000]}
    for score in scores: 
        #print score, '\n'
        grid_search = GridSearchCV(pipeline, param_grid = param, verbose = 3, cv = 5, scoring = score)
        grid_search.fit(Xtrain, ytrain)
        output[score] = grid_search.best_score_, grid_search.best_params_
    return output

## Selecting the best setup for the Random forest algorithm

In [23]:
columns_gen = dataDegree_workDummies.columns.values.tolist()
#display(columnas)

variable2predict_gen = 'haEstadoDesempleado'

columns_gen.remove(variable2predict_gen)
#display(columnas)

features_gen = pd.Index(columns_gen)

In [24]:
y_gen, _gen = pd.factorize(train_gen[variable2predict_gen])
print(y_gen)
scoresRF_results_gen = grid_search_scores_rf(train_gen[features_gen], y_gen)
print(scoresRF_results_gen)

[0 0 0 ..., 0 0 1]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.645462 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.482143 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.497671 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.606061 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.644911 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5, score=0.657099 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5, score=0.511646 -   0.1s
[CV] randomforestclassifier__max_features=auto,

[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, score=0.591298 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, score=0.653458 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.660202 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.579193 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.340839 -   0.1s
[CV] randomforestclassifier__max_features=sqrt,

[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=1, score=0.445652 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=1, score=0.575758 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=1, score=0.650350 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.659426 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.608696 -   0.1s
[CV] randomforestclassifier__max_features=log2,

...

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.555778 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=1, score=0.604009 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5, score=0.495778 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5, score=0.323159 -   0.1s
[CV] randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=auto, randomforestclassifier__min_samples_leaf=5, score=0.439801 -   0.1s
[CV] randomforestclassifier__max_features=auto,

[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, score=0.613460 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.479981 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.356776 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.398634 -   0.1s
[CV] randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=5, score=0.584401 -   0.1s
[CV] randomforestclassifier__max_features=sqrt,

[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.512058 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.345054 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.461033 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.556797 -   0.1s
[CV] randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5 
[CV]  randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_leaf=5, score=0.634894 -   0.1s
[CV] randomforestclassifier__max_features=log2,

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    7.1s finished


## Obtaining the best Random Forest configuration through the output of the benchmarks previously computed

In [25]:
best_RF_values_gen = max(scoresRF_results_gen.items(), key=lambda a: a[1])
if best_RF_values_gen[1][0] == 1.0:
    best_RF_values_gen = sorted(scoresRF_results_gen.iteritems(),key=lambda v: v[1],reverse=True)[1]

    
print(best_RF_values_gen)
    
#Accessing to the config values within the dict inside the list contained in the defaultdict
rfC__min_samples_leaf_gen = best_RF_values_gen[1][1]['randomforestclassifier__min_samples_leaf']
rfC__max_features_gen = best_RF_values_gen[1][1]['randomforestclassifier__max_features']

('accuracy', (0.6628358440751669, {'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__min_samples_leaf': 200}))


## Run the Random Forest with the best setup found

In [26]:
best_randomforest_gen = RandomForestClassifier(min_samples_leaf = rfC__min_samples_leaf_gen, max_features = str(rfC__max_features_gen))
best_randomforest_gen.fit(train_gen[features_gen], y_gen)
None

## Test the trained Random Forest

In [27]:
target_names_gen = np.array([True, False])

In [28]:
preds_gen = target_names_gen[best_randomforest_gen.predict(test_gen[features_gen])]

In [29]:
pd.crosstab(test_gen[variable2predict_gen], preds_gen, rownames=['actual'], colnames=['preds'])

preds,False,True
actual,,
0,125,941
1,46,2193


In [31]:
print(metrics.classification_report(test_gen[variable2predict_gen], preds_gen))

             precision    recall  f1-score   support

          0       0.73      0.12      0.20      1066
          1       0.70      0.98      0.82      2239

avg / total       0.71      0.70      0.62      3305



## Identification of the most important features for the model

In [32]:
importances_gen = best_randomforest_gen.feature_importances_
feat_labels_gen = dataDegree_workDummies.columns[1:]
indices_gen = np.argsort(importances_gen)[::-1]

importances_threshold_gen = 0.05

listImportances_gen = []
listNonImportant_gen = []

for f in range(train_gen[features_gen].shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            features_gen[indices_gen[f]], 
                            importances_gen[indices_gen[f]]))
    if importances_gen[indices_gen[f]] > importances_threshold_gen:
        listImportances_gen.append(features_gen[indices_gen[f]])
    else:
        listNonImportant_gen.append(features_gen[indices_gen[f]])

 1) universidad_id                 0.437347
 2) otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal 0.150274
 3) sexo_id                        0.107880
 4) residenciaExtranjeroMotivos_cod 0.097754
 5) otrosCriteriosSeleccionarPuestoTrabajo_prestigio 0.075377
 6) titulacion_id                  0.054354
 7) otrosCriteriosSeleccionarPuestoTrabajo_salario 0.032414
 8) otrosCriteriosSeleccionarPuestoTrabajo_relacionEstudios 0.016768
 9) residenciaExtranjeroDespuesEstudios_trabajo 0.011884
10) cicloTitulacion_id             0.007722
11) otrosCriteriosSeleccionarPuestoTrabajo_geografica 0.004036
12) otrosCriteriosSeleccionarPuestoTrabajo_otros 0.001992
13) residenciaExtranjeroDuranteEstudiosMotivos_cod 0.001354
14) residenciaExtranjeroDespuesEstudios 0.000537
15) tipologiaPracticas             0.000306
16) residenciaExtranjeroDespuesEstudios_estudios 0.000000
17) residenciaExtranjeroDuranteEstudios 0.000000
18) residenciaExtranjeroDuranteEstudios_estudios 0.000000
19) residenciaExt

In [33]:
print("Most important features for the model:")
display(listImportances_gen)

Most important features for the model:


['universidad_id',
 'otrosCriteriosSeleccionarPuestoTrabajo_circunstanciasPersonal',
 'sexo_id',
 'residenciaExtranjeroMotivos_cod',
 'otrosCriteriosSeleccionarPuestoTrabajo_prestigio',
 'titulacion_id']